# Driving through the maze of traffic cones - Data Collection

If you ran through the basic motion and collision avoidance notebooks, hopefully you're enjoying how easy it can be to make your Jetbot move around and avoid the obstacles like walls or chairs. In this notebook we will show how to teach JetBot *drive through the maze of traffic cones*!  

Similar to collision avoidance notebook, we are going to attempt to solve the problem using deep learning classification model. The idea is to create a virtual "safety bubble" around the robot. Within this safety bubble, the robot is able to spin in a circle without hitting any objects. However, unlike collision avoidance example with only two classes (free or blocked) our classifier with try to determine which way to spin to stay on a path and by how much. The output of our model is a set of four probabilities:
- **p(Left)** - a probability of turning left (spinning counterclockwise)
- **p(right)** - a probability of turning right (spinning clockwise)
- **p(blocked)** - a probability of the path being blocked
- **p(free)** - a probability of no obstacles in front of the robot (so it is safe to move forward)

This is how we collect the data:  

First, we'll manually place the robot in scenarios where it's "safety bubble" is violated, and label these scenarios ``blocked``.  We save a snapshot of what the robot sees along with this label.

Second, we'll manually place the robot in scenarios where it's safe to move forward a bit, and label these scenarios ``free``.  Likewise, we save a snapshot along with this label.

Thrird, we'll manually place the robot in scenarios where spinning to the left (counterclockwise) would be the optimal move and label these scenarios ``left``. Likewise, we save a snapshot along with this label. Try to vary the angle of the desired rotation - place the robot in scenarios where this angle is larger or smaller.

Finally, we'll manually place the robot in scenarios where turning right (clockwise) would be the optimal move and label these scenarios ``right``. Likewise, we save a snapshot along with this label. Try to vary the angle of the desired rotation - place the robot in scenarios where this angle is larger or smaller. 

Once we have 100-200 images and labels for each of four classes we have to options we can 
1. Upload this data to a GPU enabled machine where we'll *train* a neural network to predict the probabilities above  based off of the image it sees, OR
2. Train model on JetBot's GPU  

We'll use our trained model to implement a simple driving behavior in the end :)

> IMPORTANT NOTE:  When JetBot spins in place, it actually spins about the center between the two wheels, not the center of the robot chassis itself.  This is an important detail to remember when you're trying to estimate whether the robot's safety bubble is violated or not. If in doubt it's better to lean on the cautious side (a big safety bubble).  We want to make sure JetBot doesn't enter a scenario that it couldn't get out of by turning in place.

### Display live camera feed

First, let's initialize and display our camera like we did in the *teleoperation* notebook.  

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

After running the cell above JetBot should display the camera feed. 

If you observe a substantial time lag, there are a couple of things you can try:

1. Make sure you use the listed battery pack and it is charged

2. Check if Jetson Nano is in MAXN or 5W mode. You can determine this by calling the following from a terminal:

``nvpmodel -q``

3. Move your JetBot closer to WiFi router

4. Restart nvargus-daemon (details are [here](https://github.com/NVIDIA-AI-IOT/jetbot/issues/47)):

``sudo systemctl restart nvargus-daemon``

5. Shutdown all kernels (*Kernel -> Shudown All Kernels*) and restart the notebook Move your JetBot closer to WiFi router

6. Stop and disable [rsyslog](https://github.com/NVIDIA-AI-IOT/jetbot/issues/63):

``sudo service rsyslog stop``

``sudo systemctl disable rsyslog``

If nothing helps, you should still be able to collect the data - you just cannot rely on the display with the camera feed which is ammoying but not critical.)  

Now let's create a few directories where we'll store all our data.  We'll create a folder ``dataset_cones`` that will contain four sub-folders ``free``, ``left``, ``right`` and ``blocked`` where we'll place the images for each scenario.

In [2]:
import os

right_dir = 'dataset_cones/right'
left_dir = 'dataset_cones/left'
free_dir = 'dataset_cones/free'
blocked_dir = 'dataset_cones/blocked'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(right_dir)
    os.makedirs(left_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about the same number of images for each class (``free``, ``left``, ``right`` or ``blocked``.)  It also helps to know how many images we've collected overall.

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free',   button_style='success', layout=button_layout)
right_button = widgets.Button(description='add right', button_style='info', layout=button_layout)
left_button = widgets.Button(description='add left',   button_style='warning', layout=button_layout)
blocked_button = widgets.Button(description='add blocked',   button_style='danger', layout=button_layout)

free_count = widgets.IntText(layout=button_layout,  value=len(os.listdir(free_dir)))
right_count = widgets.IntText(layout=button_layout, value=len(os.listdir(right_dir)))
left_count = widgets.IntText(layout=button_layout,  value=len(os.listdir(left_dir)))
blocked_count = widgets.IntText(layout=button_layout,  value=len(os.listdir(blocked_dir)))

# display(widgets.HBox([right_count, right_button]))
# display(widgets.HBox([free_count, free_button]))
# display(widgets.HBox([left_count, left_button]))
# display(widgets.HBox([blocked_count, blocked_button]))

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_right():
    global right_dir, right_count
    save_snapshot(right_dir)
    right_count.value = len(os.listdir(right_dir))

def save_left():
    global left_dir, left_count
    save_snapshot(left_dir)
    left_count.value = len(os.listdir(left_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))

# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
right_button.on_click(lambda x: save_right())
left_button.on_click(lambda x: save_left())
blocked_button.on_click(lambda x: save_blocked())

Great! Now the buttons above should save images to the ``free``,  ``left`` and ``right`` directories.  You can use the Jupyter Lab file browser to view these files!

Now go ahead and collect some data 

1. Place the robot in a scenario where it's supposed to turn right and press ``add right``
2. Place the robot in a scenario where it's supposed to turn left and press ``add left``
3. Place the robot in a scenario where it's free and press ``add free``
3. Place the robot in a scenario where it's blocked and press ``add blocked``
5. Repeat 1, 2, 3, 4

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below

Here are some tips for labeling data

1. Try different orientations (e.g. sharp right vs slight right, closer to the cone or further away from it, etc.) 
2. Try different lighting
3. Try different textured floors / objects;  patterned, smooth, glass, etc.

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.  It's important
to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).  But don't worry, it goes pretty fast once you get going :)

In [5]:
display(image)
# display(widgets.HBox([right_count, right_button]))
# display(widgets.HBox([free_count, free_button]))
# display(widgets.HBox([left_count, left_button]))
# display(widgets.HBox([blocked_count, blocked_button]))

# display buttons
middle_box = widgets.VBox([free_button, blocked_button])
controls_box = widgets.HBox([left_button, middle_box, right_button])
display(controls_box)

# display counts
middle_box_count = widgets.VBox([free_count, blocked_count])
controls_box_count = widgets.HBox([left_count, middle_box_count, right_count])
display(controls_box_count)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [6]:
!zip -r -q dataset_cones.zip dataset_cones

You should see a file named ``dataset_cones.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

> NOTE: If Chrome blocks download from the notebook you can use Internet Explorer for downloading (i.e. connect to JetBot from Internet Explorer and download the data) 

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model_cones.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!